# nlp-project-vqa


In [1]:
import mindspore
import numpy as np
import os
from easydict import EasyDict
from preprocess.preprocess import *
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"  # 允许重复载入lib文件

In [ ]:
import moxing as mox
# 请替换成自己的obs路径
mox.file.copy_parallel(src_url="s3://nlp-haofeng/nlp_project_vqa", dst_url='./') 

In [2]:
from mindspore import context
context.set_context(mode=context.GRAPH_MODE)

In [2]:
from mindspore import context
context.set_context(mode=context.PYNATIVE_MODE)

## 1 预处理

### 1.1 预处理配置

In [3]:
padding = '<pad>'
config = EasyDict({
	'train_ans_path': './data/annotations/train.json',
	'train_que_path': './data/questions/train.json',
	'valid_ans_path': './data/annotations/val.json',
	'valid_que_path': './data/questions/val.json',
	'test_ans_path':  './data/annotations/test.json',
	'test_que_path':  './data/questions/test.json',
	'train_img_path': './data/images/train/COCO_train2014_',
	'test_img_path': './data/images/test/COCO_val2014_',
	'val_img_path': './data/images/val/COCO_val2014_',
	'max_length': 25,
	'dict_path': './mindrecord/dict.npy',
	'idx_word_dict_path': './mindrecord/idx_word_dict.npy',
	'num_splits': 1,
	'train_mindrecord_path': './mindrecord/train.mindrecord',
	'valid_mindrecord_path': './mindrecord/valid.mindrecord',
	'test_mindrecord_path':  './mindrecord/test.mindrecord',
})

### 1.2 读取数据

注: 只取那些答案长度为1的vqa组合

In [4]:
# get 3 types of input data
train_images, train_questions, train_answers = get_list(config.train_que_path, config.train_ans_path)
valid_images, valid_questions, valid_answers = get_list(config.valid_que_path, config.valid_ans_path)
test_images,  test_questions,  test_answers  = get_list(config.test_que_path,  config.test_ans_path)

In [5]:
total_questions = train_questions + valid_questions + test_questions
total_answers = train_answers + valid_answers + test_answers

### 1.3 构建词典

In [6]:
# build word vocab
word_dict = dict({'<pad>': 0})
word_dict = add_word_into_dict(total_questions, word_dict)
word_dict = add_word_into_dict(total_answers, word_dict)

In [7]:
# build revert dict
idx_word_dict = dict()
for item in word_dict.items():
	idx_word_dict[item[1]] = item[0]

In [8]:
# save dict
np.save(config.dict_path, word_dict)
np.save(config.idx_word_dict_path, idx_word_dict)

### 1.4 向量化 & 补齐长度

In [9]:
# word -> vector & padding
train_questions_vec = get_vec_and_pad(train_questions, word_dict, config.max_length)
valid_questions_vec = get_vec_and_pad(valid_questions, word_dict, config.max_length)
test_questions_vec = get_vec_and_pad(test_questions, word_dict, config.max_length)

train_answers_vec = get_vec_and_pad(train_answers, word_dict, 1)
valid_answers_vec = get_vec_and_pad(valid_answers, word_dict, 1)
test_answers_vec = get_vec_and_pad(test_answers, word_dict, 1)


train_images_list = read_image(train_images, config.train_img_path)
np.save('./mindrecord/train_images_list', train_images_list)

valid_images_list = read_image(valid_images, config.val_img_path)
np.save('./mindrecord/valid_images_list', valid_images_list)

test_images_list = read_image(test_images, config.test_img_path)
np.save('./mindrecord/test_images_list', test_images_list)

### 1.5 生成MindRecord

In [10]:
generate_mindrecord(config.train_mindrecord_path, config.num_splits, train_images_list, train_questions_vec, train_answers_vec)
generate_mindrecord(config.valid_mindrecord_path, config.num_splits, valid_images_list, valid_questions_vec, valid_answers_vec)
generate_mindrecord(config.test_mindrecord_path,  config.num_splits, test_images_list,  test_questions_vec, test_answers_vec)

train
valid
test


## 2 加载数据

### 2.1 加载词典

In [4]:
# load dict
word_dict = np.load(config.dict_path, allow_pickle=True).item()
idx_word_dict = np.load(config.idx_word_dict_path, allow_pickle=True).item()

### 2.2 训练配置

In [ ]:
model_name = 'baseline'

In [5]:
train_config = EasyDict({
	'model': model_name,
	'vocab_size': 10233,
	'batch_size': 128,
	'epoch_size': 3,
	'max_length': 25,
	'hidden_size': 1024,
	'lr': 1e-3,
	'momentum': 0.9,
	'early_stop': 100,
	# 'save_checkpoint_steps': 1279,
	'ckpt_save_path': './ckpt',
	'checkpoint_path': './ckpt/'+model_name+'.ckpt',
	# 'keep_checkpoint_max': 2,
})

### 2.3 生成数据集

In [6]:
# create dataset
train_dataset = generate_dataset(config.train_mindrecord_path, train_config.batch_size, train_config.epoch_size)
valid_dataset = generate_dataset(config.valid_mindrecord_path, train_config.batch_size, train_config.epoch_size)
test_dataset  = generate_dataset(config.test_mindrecord_path , train_config.batch_size, 1)

## 3 训练模型

### 3.1 创建模型

In [7]:
import mindspore.nn as nn
import mindspore.ops.operations as P
from utils.metric_utils import *
from utils.wrapper_utils import *
from utils.callback_utils import *

In [8]:
class Network(nn.Cell):
	def __init__(self, train_config):
		super(Network, self).__init__()
		self.reshape = P.Reshape()
		self.embedding = nn.Embedding(train_config.vocab_size, train_config.hidden_size)
		self.out = nn.Dense(train_config.hidden_size*train_config.max_length, train_config.vocab_size)
	def construct(self, images, questions):
		x = self.embedding(questions)
		x = x.reshape(x.shape[0], -1)
		x = self.out(x)
		return x

In [9]:
# 创建网络
network = Network(train_config)
network = LossAndAccWrapper(network, train_config)

### 3.2 开始训练

In [10]:
def train(network, train_dataset, valid_dataset, train_config):
	# 创建文件夹
	if not os.path.exists(train_config.ckpt_save_path):
		os.mkdir(train_config.ckpt_save_path)
	
	# 创建模型
	model = mindspore.Model(network, eval_network=network, metrics={'acc': DummyAccuracyMetric()})
	
	# 获取回调函数
	train_callbacks, _ = get_network_callbacks(model, train_dataset, valid_dataset, train_config)

	# 训练，保留最好模型
	model.train(train_config.epoch_size, train_dataset, callbacks=train_callbacks, dataset_sink_mode=True)


In [ ]:
train(network, train_dataset, valid_dataset, train_config)

## 4 测试模型

### 4.1 创建测试模型

In [ ]:
from mindspore.train.serialization import load_checkpoint

In [ ]:
network = Network(train_config)
load_checkpoint(train_config.checkpoint_path, net=network)
network = LossAndAccWrapper(network, train_config)

### 4.2 开始测试

In [ ]:
def test(network, test_dataset):
	# 创建测试模型
	model = mindspore.Model(network, metrics={'acc': DummyAccuracyMetric()})

	# 回调显示
	test_callbacks = [TestCallback()]

	# 生成结果
	model.eval(test_dataset, callbacks=test_callbacks, dataset_sink_mode=True)

In [ ]:
test(network, test_dataset)